### Part II: Motion Detection Using Optical Flow
Completed by: Liliya Panfilova

In [4]:
import cv2
import numpy as np
from datetime import datetime

In [9]:
# Initialize the counter for saved images
counter = 0
save_counter = 0

# Start capturing video from the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Get first frame, convert to grayscale
ret, frame1 = cap.read()
if not ret:
    raise IOError("Cannot read from webcam")
previous_frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

while True:
    ret, frame2 = cap.read()
    if not ret:
        break
    
    # Get the second frame in grayscale
    current_frame = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Farneback Optical Flow parameters
    method = cv2.calcOpticalFlowFarneback
    params = [0.5, 3, 15, 3, 5, 1.2, 0]

    # Calculate Optical Flow
    flow = method(previous_frame, current_frame, None, *params)

    # Convert optical flow into Polar coordinates to get magnitude
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Use a threshold, to only count the significant ones
    mag_thresholded = (mag > 20)
    percentage = mag_thresholded.sum()/(frame_width * frame_height)

    # Convert the angle to hue
    ang_degrees = ang * 180 / np.pi / 2

    # Scale angle and magnitude: 0 to 255
    ang_scaled = cv2.normalize(ang_degrees, None, 0, 255, cv2.NORM_MINMAX)
    mag_scaled = cv2.normalize(mag_thresholded.astype(np.float32), None, 0, 255, cv2.NORM_MINMAX)

    # Visualize the optical flow vectors
    mask = np.zeros_like(frame2)
    mask[..., 0] = ang_scaled
    mask[..., 1] = 255
    mask[..., 2] = mag_scaled

    # Convert HSV image into BGR for demo
    bgr = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    
    # Draw arrows to represent displacement vectors
    for i in range(0, frame2.shape[0], 10):
        for j in range(0, frame2.shape[1], 10):
            dx = int(flow[i, j, 0])
            dy = int(flow[i, j, 1])
            cv2.arrowedLine(bgr, (j, i), (j + dx, i + dy), (0, 255, 0), 1)

    # Check for significant changes in the image
    if percentage > 0.015:  # Change 1.0 to the desired sensitivity
        print(f"Percentage of significant optical flow: {percentage * 100:.2f}%")
        print(f"Activity detected at: {datetime.now()}")
        # Save the current color frame with timestamp
        save_counter += 1
        filename = f"frame_{save_counter}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
        cv2.imwrite(f'opticalflow_{save_counter}.jpg', bgr)
        print(f"Saved {filename}")
        # Update the background image
        previous_frame = current_frame
        # Wait for 5 seconds
        cv2.waitKey(5000)
        
    # Display the frame with optical flow vectors and displacement arrows
    cv2.imshow('Optical Flow and Displacement Vectors', bgr)
    # Exit the program by entering esc or 'q'
    key = cv2.waitKey(1)
    if key in [27, ord('q')]:  # 27 is the escape key
        break
    
    # Capture a frame image after pressing x
    if key & 0xFF == ord('x'):
        
        image_name = 'image' + str(counter) + '.png'
        cv2.imwrite(image_name, bgr)
        counter += 1

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

Percentage of significant optical flow: 5.72%
Activity detected at: 2023-11-15 22:48:14.394233
Saved frame_1_20231115_224814.jpg
Percentage of significant optical flow: 6.94%
Activity detected at: 2023-11-15 22:48:19.515700
Saved frame_2_20231115_224819.jpg
Percentage of significant optical flow: 4.32%
Activity detected at: 2023-11-15 22:48:24.619090
Saved frame_3_20231115_224824.jpg
Percentage of significant optical flow: 4.30%
Activity detected at: 2023-11-15 22:48:29.738328
Saved frame_4_20231115_224829.jpg
Percentage of significant optical flow: 2.72%
Activity detected at: 2023-11-15 22:48:34.851538
Saved frame_5_20231115_224834.jpg
Percentage of significant optical flow: 2.63%
Activity detected at: 2023-11-15 22:48:39.989079
Saved frame_6_20231115_224839.jpg
Percentage of significant optical flow: 2.30%
Activity detected at: 2023-11-15 22:48:41.486419
Saved frame_7_20231115_224841.jpg
Percentage of significant optical flow: 10.18%
Activity detected at: 2023-11-15 22:48:45.692582
S